### Week 3 Day 5 Exercise: Multilingual Text Summarizer with Voice Output

This Gradio application:
1. Accepts bulk text input or uploaded text files (max 1MB / 500 words)
2. Automatically detects the source language
3. Summarizes the text in your selected target language (12 languages supported)
4. Optionally reads the summary aloud using text-to-speech  


In [ ]:
import os
import gradio as gr
import tempfile
from dotenv import load_dotenv
from openai import OpenAI
from langdetect import detect, DetectorFactory
from gtts import gTTS

In [ ]:
# Load environment variables
load_dotenv(override=True)
openrouter_url = "https://openrouter.ai/api/v1"
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
if not openrouter_api_key:
    raise ValueError("OPENROUTER_API_KEY is not set in the environment variables")
else:
    print("OPENROUTER_API_KEY is ok")


In [ ]:
# Initialize
openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
ANTROPIC_MODEL = "anthropic/claude-3.5-sonnet"
MAX_FILE_SIZE_BYTES = 1 * 1024 * 1024  # 1MB
MAX_WORD_COUNT = 500

In [ ]:
SUPPORTED_LANGUAGES = {
    "en": "English",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "it": "Italian",
    "pt": "Portuguese",
    "zh-cn": "Chinese (Simplified)",
    "ja": "Japanese",
    "ko": "Korean",
    "ar": "Arabic",
    "hi": "Hindi",
    "ru": "Russian"
}

LANGUAGE_TO_CODE = {v: k for k, v in SUPPORTED_LANGUAGES.items()}

In [ ]:
system_prompt = """You are a multilingual text summarizer and translator.
Given text in any language, provide a concise summary in the specified target language.
Keep the summary clear, informative, and under 150 words.
Always respond with ONLY the summary text, no additional commentary or labels."""


In [ ]:
def validate_input(text_input= None, file_input = None):
    """
    Validate and extract text from input
    """
    text = ""
    
    # Handle file input
    if file_input is not None:
        try:
            file_size = os.path.getsize(file_input)
            if file_size > MAX_FILE_SIZE_BYTES:
                return "", f"File size ({file_size / (1024*1024):.2f}MB) exceeds 1MB limit."
            
            with open(file_input, 'r', encoding='utf-8') as f:
                text = f.read()
        except UnicodeDecodeError:
            return "", "Could not read file. Please upload a valid text file with UTF-8 encoding."
        except Exception as e:
            return "", f"Error reading file: {str(e)}"
    
    # Handle text input
    elif text_input and text_input.strip():
        text = text_input.strip()
    else:
        return "", "Please provide text input or upload a file."
    
    # Validate word count
    word_count = len(text.split())
    if word_count > MAX_WORD_COUNT:
        return "", f"Text contains {word_count} words, which exceeds the {MAX_WORD_COUNT} word limit."
    
    if word_count < 50:
        return "", "Text is too short. Please provide at least 50 words for summarization."
    
    return text, ""

In [ ]:
def detect_language(text):
    """
    Detect the language of the input text.
    """
    DetectorFactory.seed = 0 
    try:
        detected_code = detect(text)
        language_name = SUPPORTED_LANGUAGES.get(
            detected_code, f"Unknown ({detected_code})"
        )
        return detected_code, language_name
    except Exception as e:
        return "en", "English (detection failed, defaulting)"

In [ ]:
def summarize_text(text, target_language):
    """
    Summarize the input text in the target language using streaming.
    """
    user_message = f"""Please summarize the following text in {target_language}:

        {text}

    Provide a clear, concise summary in {target_language}."""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    response_stream = openrouter.chat.completions.create(
        model=ANTROPIC_MODEL,
        messages=messages,
        stream=True
    )
    
    response = ""
    for chunk in response_stream:
        delta = chunk.choices[0].delta.content or ""
        response += delta
        yield response

In [ ]:
def text_to_speech(text, lang_code):
    """
    Convert text to speech using gTTS.
    Returns: path to the generated audio file
    """
    # Map some language codes for gTTS compatibility
    gtts_lang_map = {
        "zh-cn": "zh-CN",
        "zh-tw": "zh-TW",
    }
    gtts_code = gtts_lang_map.get(lang_code, lang_code)
    
    try:
        tts = gTTS(text=text, lang=gtts_code, slow=False)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            tts.save(fp.name)
            return fp.name
    except Exception as e:
        print(f"Text to speech Error has occurred: {e}")
        return None

In [ ]:
def process_text(text_input, file_input, target_language, enable_voice):
    """
    Main processing function that orchestrates validation, summarization, and TTS.
    """
    # Input Validation
    extracted_text, error = validate_input(text_input, file_input)
    if error:
        yield error, "", None, gr.update(visible=False), gr.update(interactive=True, value="Summarize")
        return

    # Detect source language
    language_code, language_name = detect_language(extracted_text)
    detected_display = f"Detected: {language_name}({language_code})"

    # Get target language code
    target_code = LANGUAGE_TO_CODE.get(target_language, "en")

    # Summarize - disable button during summarization
    summary = ""
    for language_summary in summarize_text(extracted_text, target_language):
        summary = language_summary
        yield detected_display, summary, None, gr.update(visible=False), gr.update(interactive=False, value="Summarizing...")

    # Generate TTS if requested
    audio_path = None
    if enable_voice and summary:
        # Show processing state for voice generation
        yield detected_display, summary, None, gr.update(visible=False), gr.update(interactive=False, value="Generating voice...")
        audio_path = text_to_speech(summary, target_code)
        # Keeps the button disabled while audio plays
        yield detected_display, summary, audio_path, gr.update(visible=True, autoplay=True), gr.update(interactive=False, value="Playing audio...")
    else:
        yield detected_display, summary, None, gr.update(visible=False), gr.update(interactive=True, value="Summarize")


In [ ]:
def on_audio_end():
    """Re-enable the summarize button when audio finishes playing."""
    return gr.update(interactive=True, value="Summarize")


def on_file_upload(file):
    """Clear text input when a file is uploaded."""
    if file is not None:
        return ""
    return gr.update()

In [ ]:
# --- GRADIO INTERFACE ---

with gr.Blocks(title="Multilingual Text Summarizer", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Multilingual Text Summarizer with Voice Output")
    gr.Markdown("Upload a text file or paste text to get a summary in your chosen language. Optionally, listen to the summary read aloud.")
    
    with gr.Row():
        with gr.Column(scale=1):
            text_input = gr.Textbox(
                label="Enter text (max 500 words)",
                placeholder="Paste your text here...",
                lines=10
            )
            file_input = gr.File(
                label="Or upload a text file (max 1MB)",
                file_types=[".txt"],
                type="filepath"
            )
            target_lang = gr.Dropdown(
                choices=list(SUPPORTED_LANGUAGES.values()),
                label="Summarize to language",
                value="English"
            )
            voice_checkbox = gr.Checkbox(
                label="Read summary aloud",
                value=False
            )
            submit_btn = gr.Button("Summarize", variant="primary")
            clear_btn = gr.Button("Clear")
        
        with gr.Column(scale=1):
            detected_lang = gr.Textbox(
                label="Source Language",
                interactive=False
            )
            summary_output = gr.Markdown(label="Summary")
            audio_output = gr.Audio(
                label="Audio Summary",
                visible=False,
                autoplay=True
            )
    
    # Clear text input when file is uploaded
    file_input.change(
        fn=on_file_upload,
        inputs=[file_input],
        outputs=[text_input]
    )
    
    submit_btn.click(
        fn=process_text,
        inputs=[text_input, file_input, target_lang, voice_checkbox],
        outputs=[detected_lang, summary_output, audio_output, audio_output, submit_btn]
    )
    
    # Re-enable button when audio finishes playing
    audio_output.stop(
        fn=on_audio_end,
        inputs=[],
        outputs=[submit_btn]
    )
    
    clear_btn.click(
        fn=lambda: ("", None, "English", False, "", "", None, gr.update(interactive=True, value="Summarize")),
        inputs=[],
        outputs=[text_input, file_input, target_lang, voice_checkbox, detected_lang, summary_output, audio_output, submit_btn]
    )

demo.launch(inbrowser=True)
